In [30]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

VERSION = '99_v1_01_16'
os.makedirs('result', exist_ok=True)

fold_list = [0]

for fold in fold_list:
    model_path_list = [
        f'../../01_Baseline/exp/result/01_v1_27/pseudo_fold{fold}.csv', # deberta-v3-base
        f'../../01_Baseline/exp/result/01_v1_43/pseudo_fold{fold}.csv', # deberta-v3-large
        f'../../07_Embeddings/exp/result/07_v1_01/pseudo_df_fold{fold}.csv', # deberta-v3-large
    ]

    df_list = [
        pd.read_csv(model_path) for model_path in model_path_list
    ]

    base_df = oof_df_list[0]
    if 'text_id' not in base_df.columns:
        base_df['text_id'] = base_df['id'].values

    for df in df_list[1:]:
        try:
            df = base_df[['text_id']].merge(df, how='left', on='text_id')
        except:
            df['text_id'] = df['id'].values
            df = base_df[['text_id']].merge(df, how='left', on='text_id')


    num_models = len(model_path_list)

    TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

    preds = 0
    for df in df_list:
        preds += df[TARGET_COLS].values / num_models

    preds = []
    for df in df_list:
        preds.append(df[TARGET_COLS].values)

    preds = np.stack(preds, axis=0)
    preds = preds.mean(axis=0)
    print(preds.shape)

    for i,col in enumerate(TARGET_COLS):
        base_df[col] = preds[:,i]
    
    base_df.to_csv(f'result/{VERSION}_pseudo_fold{fold}.csv', index=False)

(15142, 6)
